In [1]:
!pip install wandb -q
import wandb
wandb.login()


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


wandb: Currently logged in as: ahmadhakimiadnan (ahmadhakimiadnan-other) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
pip install torch bitsandbytes transformers accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print(torch.cuda.get_device_name(0))  # Should show "RTX 4070"
print(torch.cuda.get_device_properties(0).total_memory / 1e9)  # Should show ~12GB

NVIDIA GeForce RTX 4070 Ti
12.479430656


In [3]:
torch.cuda.empty_cache()

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
pip install accelerate bitsandbytes


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig
import wandb

wandb.init(project="fine-tuning-llms")

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
print("model is loaded")

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

model is loaded


In [3]:
for name, param in model.named_parameters():
    print(name, param.shape)

model.embed_tokens.weight torch.Size([128256, 3072])
model.layers.0.self_attn.q_proj.weight torch.Size([3072, 3072])
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 3072])
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 3072])
model.layers.0.self_attn.o_proj.weight torch.Size([3072, 3072])
model.layers.0.mlp.gate_proj.weight torch.Size([8192, 3072])
model.layers.0.mlp.up_proj.weight torch.Size([8192, 3072])
model.layers.0.mlp.down_proj.weight torch.Size([3072, 8192])
model.layers.0.input_layernorm.weight torch.Size([3072])
model.layers.0.post_attention_layernorm.weight torch.Size([3072])
model.layers.1.self_attn.q_proj.weight torch.Size([3072, 3072])
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 3072])
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 3072])
model.layers.1.self_attn.o_proj.weight torch.Size([3072, 3072])
model.layers.1.mlp.gate_proj.weight torch.Size([8192, 3072])
model.layers.1.mlp.up_proj.weight torch.Size([8192, 3072])
model.layer

In [5]:
#freeze all parameters first
for param in model.parameters():
    param.requires_grad = False

#unfreeze top n layers + lm_head
n_layers = 2  
layers = model.model.layers

for layer in layers[-n_layers:]:
    for param in layer.parameters():
        param.requires_grad = True

#unfreeze final lm_head
for param in model.lm_head.parameters():
    param.requires_grad = True

#print trainable parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} | Total params: {total:,} ({100 * trainable/total:.4f}%)")
wandb.log({"trainable_params": trainable, "total_params": total})


Trainable params: 595,341,312 | Total params: 3,212,749,824 (18.5306%)


In [4]:
for name, param in model.lm_head.named_parameters():
    print(name, param.shape, param.dtype, param.requires_grad)

weight torch.Size([128256, 2048]) torch.float32 True


In [5]:
class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} | all params: {all_param} | trainable%: {100 * trainable_params / all_param:.2f}")

print_trainable_parameters(model)

trainable params: 384311296 | all params: 1235814400 | trainable%: 31.10


In [8]:
pip install pandas datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 95.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 97.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

df = pd.read_csv("CMNEE.csv")
df = df[df['instruction'].notnull() & df['new_output'].notnull()]

if 'input' not in df.columns:
    df['input'] = ''

#split
train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

#HF DatasetDict
data = DatasetDict({
    "train": Dataset.from_pandas(train_df, preserve_index=False),
    "validation": Dataset.from_pandas(val_df, preserve_index=False),
    "test": Dataset.from_pandas(val_df.copy(), preserve_index=False)
})

In [10]:
#prompt
def create_prompt(example):
    prompt = f"Instruction:\n{example['instruction']}\n\n"
    if example['input']:
        prompt += f"Input:\n{example['input']}\n\n"
    prompt += f"Response:\n{example['new_output']}"
    example["prompt"] = prompt
    return example

data = data.map(create_prompt)

Map:   0%|          | 0/2547 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoTokenizer
#tokenization
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    tokens = tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_data = data.map(tokenize_function, batched=True)
print(tokenized_data)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Map:   0%|          | 0/2547 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'new_output', 'prompt', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2547
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'new_output', 'prompt', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'new_output', 'prompt', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 450
    })
})


In [12]:
from datetime import datetime
import transformers
import os
import gc

base_model_name = "Llama3.23B"
run_name = "TopLayer3B"
output_dir = "./" + run_name

#training Arguments with Efficient Checkpoint Management
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    prediction_loss_only=True,
    num_train_epochs=3,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    output_dir=output_dir,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=1,
    eval_strategy="steps",
    eval_steps=50,
    do_eval=True,
    report_to='wandb',
    run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
)

#custom Callback to Clean Up Old Checkpoints
class CleanupCallback(transformers.TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        # Keep only the 3 most recent checkpoints (adjust as needed)
        checkpoints = []
        for entry in os.listdir(args.output_dir):
            if entry.startswith("checkpoint-"):
                checkpoints.append(entry)
        checkpoints.sort(key=lambda x: int(x.split("-")[1]))
        
        # Delete all but the newest checkpoint
        for checkpoint in checkpoints[:-1]:
            os.system(f"rm -rf {os.path.join(args.output_dir, checkpoint)}")

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[CleanupCallback()]  
)

model.config.use_cache = False
torch.backends.cuda.enable_flash_sdp(True)

In [13]:
trainer.train()
del trainer
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()
wandb.finish()

Step,Training Loss,Validation Loss
50,2.638400,2.636600
100,2.424600,2.538697
150,2.172400,2.415498
200,2.240100,2.216719
250,2.023300,2.062836
300,1.962500,1.969307
350,1.887100,1.781910
400,1.557800,1.655028
450,1.456700,1.520834
500,1.339500,1.383883


eval/loss,██▇▇▆▆▅▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▃▆▅▆▇▂▁▂▇▂▁▅▆▄▄▆█▁▂▁▂▂▂▂▄▄▂▃▃▂▂▂▁▁▁▂▃▃
eval/samples_per_second,▆▃▄▃▂▇█▇▂▇█▄▃▅▅▃▁█▇█▇▇▇▇▅▅▇▆▆▇▇▇███▇▆▆
eval/steps_per_second,▆▃▄▃▂▇█▇▂▇█▄▃▅▅▃▁█▇█▇▇▇▇▅▅▇▆▆▇▇▇███▇▆▆
total_params,▁▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇███
train/global_step,▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▇█▆▆▅▅▅▆▅▆▆▆▆▆▅▄▄▆▅▅▄▃▄▃▃▃▃▂▄▃▁▁▂▂▃▁▁▁▁▂
train/learning_rate,▂▄▅██▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▁▁▁
train/loss,████▇▇▇▆▆▆▅▅▆▅▅▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
trainable_params,▁▁


In [15]:
pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 100.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 100.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
#human evaluation
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

base_model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

#load baseline model
baseline_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
baseline_model.eval()

#load fine-tuned top layer checkpoint
model = AutoModelForCausalLM.from_pretrained(
    "TopLayer3B/checkpoint-1911",
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

def build_prompt(instruction, input_text=""):
    prompt = f"Instruction:\n{instruction.strip()}\n"
    if input_text.strip():
        prompt += f"\nInput:\n{input_text.strip()}\n"
    prompt += "\nResponse:\n"
    return prompt

def generate_text(instruction, input_text):
    prompt = build_prompt(instruction, input_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    #baseline response
    with torch.no_grad():
        baseline_outputs = baseline_model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True
        )
    baseline_text = tokenizer.decode(baseline_outputs[0], skip_special_tokens=True).split("Response:")[-1].strip()

    #fine-tuned top layer response
    with torch.no_grad():
        tuned_outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True
        )
    tuned_text = tokenizer.decode(tuned_outputs[0], skip_special_tokens=True).split("Response:")[-1].strip()

    return f"Baseline Response: \n{baseline_text}\n\nTop-Layer Fine-tuned Response: \n{tuned_text}"

interface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Instruction", lines=2, placeholder="e.g. What is the best team in London?"),
        gr.Textbox(label="Input (optional)", lines=2, placeholder="Add context if needed"),
    ],
    outputs=gr.Textbox(label="Comparison"),
    title="LLaMA 3B Baseline vs Top-Layer Fine-tuned",
    description="Compare the baseline LLaMA 3B model with top-layer fine-tuned version."
)

interface.launch(share=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://5e9d10bf5da0a62864.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [18]:
pip install rouge-score bert-score nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 88.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24988 sha256=933a5ff97531c7473536342c6ea5fe517e109cfe18fc6cf970552ccc44f1c4bd
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [22]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import torch
import logging
from transformers import logging as hf_logging

from transformers import AutoTokenizer, AutoModelForCausalLM

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt')

base_model = "meta-llama/Llama-3.2-1B"
fine_tuned_checkpoint = "TopLayer3B/checkpoint-1911"

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    fine_tuned_checkpoint,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

logging.getLogger().setLevel(logging.ERROR)
hf_logging.set_verbosity_error()

# evaluation lists
true_answers = []
predicted_answers = []

for sample in tqdm(data["test"]):
    instruction = sample["instruction"]
    input_text = sample.get("input", "")
    true_output = sample["new_output"]

    prompt = f"Instruction:\n{instruction.strip()}\n"
    if input_text and input_text.strip():
        prompt += f"\nInput:\n{input_text.strip()}\n"
    prompt += "\nResponse:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=False
        )
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predicted_response = generated_text.split("Response:")[-1].strip()

    predicted_answers.append(predicted_response)
    true_answers.append(true_output.strip())

#f1 score
true_token_sets = [set(ans.lower().split()) for ans in true_answers]
pred_token_sets = [set(ans.lower().split()) for ans in predicted_answers]

all_tokens = list(set().union(*true_token_sets, *pred_token_sets))
mlb = MultiLabelBinarizer(classes=all_tokens)
y_true_bin = mlb.fit_transform(true_token_sets)
y_pred_bin = mlb.transform(pred_token_sets)

f1 = f1_score(y_true_bin, y_pred_bin, average="macro")
print(f"\nMacro F1 Score on validation set: {f1:.4f}")

#BERTScore
from bert_score import score as bert_score
print("\nCalculating BERTScore...")

P = predicted_answers
R = true_answers

P_scores, R_scores, F1_scores = bert_score(P, R, lang='en', verbose=True)
average_bert_f1 = F1_scores.mean().item()
print(f"Average BERTScore F1 on validation set: {average_bert_f1:.4f}")

#BLEU Score
print("\nCalculating BLEU Score...")

smoothie = SmoothingFunction().method4
bleu_scores = []
for ref, pred in zip(true_answers, predicted_answers):
    ref_tokens = [nltk.word_tokenize(ref.lower())]
    pred_tokens = nltk.word_tokenize(pred.lower())
    score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)
    bleu_scores.append(score)

average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score on validation set: {average_bleu:.4f}")

#ROUGE Scores
print("\nCalculating ROUGE Scores...")

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_list, rouge2_list, rougeL_list = [], [], []

for ref, pred in zip(true_answers, predicted_answers):
    scores = scorer.score(ref, pred)
    rouge1_list.append(scores['rouge1'].fmeasure)
    rouge2_list.append(scores['rouge2'].fmeasure)
    rougeL_list.append(scores['rougeL'].fmeasure)

avg_rouge1 = sum(rouge1_list) / len(rouge1_list)
avg_rouge2 = sum(rouge2_list) / len(rouge2_list)
avg_rougeL = sum(rougeL_list) / len(rougeL_list)

print(f"Average ROUGE-1 F1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 450/450 [19:07<00:00,  2.55s/it]



Macro F1 Score on validation set: 0.0640

Calculating BERTScore...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 2.45 seconds, 183.51 sentences/sec
Average BERTScore F1 on validation set: 0.8169

Calculating BLEU Score...


Average BLEU Score on validation set: 0.0327

Calculating ROUGE Scores...
Average ROUGE-1 F1: 0.1073
Average ROUGE-2 F1: 0.0668
Average ROUGE-L F1: 0.1068
